In [25]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
import scipy as sp

from HJK.basis.monomials import monomials
from sklearn.preprocessing import PolynomialFeatures

from deeptime.basis import Monomials
#import PySINDY.pysindy as ps


## Collect random datapoints from a domain

In [8]:
domain = 5
dim = 2
numIC = 1000

X = np.random.uniform(low=-domain, high=domain, size=(dim,numIC)).astype(float) 
np.shape(X)

(2, 1000)

## Monomials from `deeptime`

In [9]:
deeptime_lib = Monomials(p=2, d=2)
deeptime_lib.get_feature_names(["x1", "x2"])

['', 'x1', 'x2', 'x1^2', 'x1 x2', 'x2^2']

In [10]:
#X = torch.tensor([[1.0, 2.0], [1.0, 3.0]])
X = np.array([[1.0, 2.0]], dtype=float)
deeptime_lib(X)

array([[1., 1., 2., 1., 2., 4.]])

## Monomials from `pySINDy`

In [11]:
#sindy_lib = ps.PolynomialLibrary(degree=2)

## Monomials from d3s github

In [12]:
d3s_monomials = monomials(2)
d3s_monomials.display(np.array([1.0, 1.0, 1.0, 1.0]),2)

1.000001 + 1.00000 x_1 + 1.00000 x_2 + 1.00000 x_1^2


In [13]:
X = np.array([[1, 2], [1,3]])
obs = d3s_monomials(X)
np.shape(obs)
obs

array([[1., 1.],
       [1., 2.],
       [1., 3.],
       [1., 4.],
       [1., 6.],
       [1., 9.]])

In [14]:
Dobs = d3s_monomials.diff(X)
np.shape(obs)
obs

array([[1., 1.],
       [1., 2.],
       [1., 3.],
       [1., 4.],
       [1., 6.],
       [1., 9.]])

# Monomials from  `sklearn.PolynomialFeatures`

In [15]:
#X = torch.tensor([[1.0, 2.0], [1.0, 3.0]])
x = np.array([1.0,2.0], dtype=float)
monomial_basis = PolynomialFeatures(2)
Psi = monomial_basis.fit(X)
Psi.get_feature_names(['x1','x2'])

c:\Users\sriramk\.conda\envs\HJ_Koopman\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['1', 'x1', 'x2', 'x1^2', 'x1 x2', 'x2^2']

In [16]:
Psi_vals = monomial_basis.fit_transform(X)
Psi_vals

array([[1., 1., 2., 1., 2., 4.],
       [1., 1., 3., 1., 3., 9.]])

In [17]:
def basis(x):
    monomial_basis = PolynomialFeatures(2)
    return monomial_basis.fit(x)

# calculating the jacobian

In [18]:
def jacobian(func,initial,delta=1e-3):
  f = func
  nrow = len(f(initial))
  print(nrow)
  ncol = len(initial)
  output = np.zeros(nrow*ncol)
  output = output.reshape(nrow,ncol)
  for i in range(nrow):
    for j in range(ncol):
      ej = np.zeros(ncol)
      ej[j] = 1
      dij = (f(initial+ delta * ej)[i] - f(initial- delta * ej)[i])/(2*delta)
      output[i,j] = dij
  return output

In [19]:
def f_test(x):
  x1 = x[0]
  x2 = x[1]
  output = np.zeros(3)
  output[0] = x[0]**2
  output[1] = x[1]**2
  output[2] = x[0]*x[1]
  return output

f_test([1,2])
#np.shape(f_test([1,2]))

array([1., 4., 2.])

In [20]:
jacobian(f_test,[1,2])

3


array([[2., 0.],
       [0., 4.],
       [2., 1.]])

In [21]:
x = np.array([[1.0,2.0]], dtype=float)
Psi = PolynomialFeatures(2)
Psi.fit_transform(x).squeeze()

array([1., 1., 2., 1., 2., 4.])

In [22]:
def jacobian_sk(func,initial,delta=1e-3):
  f = func
  nrow = len(f.fit_transform(initial).squeeze())
  ncol = len(initial.squeeze())
  output = np.zeros(nrow*ncol)
  output = output.reshape(nrow,ncol)
  print(np.shape(output))
  for i in range(nrow):
    for j in range(ncol):
      ej = np.zeros(ncol)
      ej[j] = 1
      dij = (f.fit_transform(initial+ delta * ej).squeeze()[i] - f.fit_transform(initial- delta * ej).squeeze()[i])/(2*delta)
      output[i,j] = dij
  return output

In [23]:
x = np.array([[1.0,2.0]], dtype=float)
jacobian_sk(Psi, x)

(6, 2)


array([[0., 0.],
       [1., 0.],
       [0., 1.],
       [2., 0.],
       [2., 1.],
       [0., 4.]])